## Step 1: Clone Repository (Kaggle Only)

Run this cell on Kaggle to get the latest code from GitHub:

In [ ]:
#delete directory if exists
import shutil
import os
if os.path.exists('prj-2-opencv-dl-pytorch'):
    shutil.rmtree('prj-2-opencv-dl-pytorch')
# Clone repository from GitHub
!git clone https://github.com/ramabyg/prj-2-opencv-dl-pytorch.git

# Add to Python path
import sys
sys.path.insert(0, '/kaggle/working/prj-2-opencv-dl-pytorch')

print("✓ Repository cloned and added to path")

## Step 2: Import Modules

In [ ]:
from src.config import get_config
from src.datamodule import KenyanFood13DataModule
from src.model import KenyanFood13Classifier
from src.trainer import train_model
from src.utils import calculate_dataset_mean_std

print("✓ All modules imported successfully")

## Step 3: Configure Training

In [ ]:
# Get configurations (auto-detects Kaggle environment)
train_config, data_config, system_config = get_config(
    num_epochs=100,          # Adjust as needed
    batch_size=32,           # Reduced for multi-GPU (each GPU gets 16)
    learning_rate=0.001,
    use_scheduler=True,
    scheduler="cosine"
)

print(f"Training for {train_config.num_epochs} epochs")
print(f"Batch size: {train_config.batch_size} per GPU")
print(f"Device: {system_config.device}")
print(f"Output directory: {system_config.output_dir}")
print("\n⚠️  Note: With 2 GPUs, effective batch size = 16 × 2 = 32")

## Step 4: Calculate Dataset Statistics (Optional)

You can skip this and use ImageNet defaults for faster startup.

In [ ]:
# Option 1: Calculate from dataset (more accurate)
# mean, std = calculate_dataset_mean_std(
#     annotations_file=data_config.annotations_file,
#     img_dir=data_config.img_dir,
#     sample_size=None  # Use more samples on Kaggle, None means use all images
# )

# Option 2: Use ImageNet defaults (faster)
# from src.utils import get_imagenet_stats
# mean, std = get_imagenet_stats()

#Option 3: Use pre computed values (fastest)
mean = [0.5672, 0.4663, 0.3659]
std = [0.2484, 0.2561, 0.2600]

## Step 5: Create Data Module and Model

In [ ]:
# Create data module
data_module = KenyanFood13DataModule(data_config, mean=mean, std=std)
data_module.setup()

print(f"✓ Data module created with {data_module.num_classes} classes")

# Create model
model = KenyanFood13Classifier(train_config, data_module.num_classes)

print(f"✓ Model created: {train_config.model_name}")

## Step 6: Train!

In [ ]:
# Train the model
trained_model, _, checkpoint_callback = train_model(
    training_config=train_config,
    data_config=data_config,
    system_config=system_config,
    model=model,
    data_module=data_module
)

print(f"\n✓ Training complete!")
print(f"Best model: {checkpoint_callback.best_model_path}")
print(f"Best accuracy: {checkpoint_callback.best_model_score:.4f}")

## Step 7: Save Outputs to Kaggle Dataset

This will save your trained model and training summary to a Kaggle Dataset for easy access later.

In [ ]:
import json
import shutil
from pathlib import Path

# Create a clean output directory for the dataset
dataset_dir = Path("/kaggle/working/kenyan_food_model_output")
if dataset_dir.exists():
    shutil.rmtree(dataset_dir)
dataset_dir.mkdir(exist_ok=True)

# Copy the best checkpoint
best_checkpoint = Path(checkpoint_callback.best_model_path)
if best_checkpoint.exists():
    shutil.copy(best_checkpoint, dataset_dir / "best_model.ckpt")
    print(f"✓ Saved best checkpoint: {best_checkpoint.name}")

# Save training summary as JSON
summary = {
    "best_val_accuracy": float(checkpoint_callback.best_model_score),
    "num_epochs": train_config.num_epochs,
    "batch_size": train_config.batch_size,
    "learning_rate": train_config.learning_rate,
    "model": train_config.model_name,
    "optimizer": train_config.optimizer,
    "scheduler": train_config.scheduler if train_config.use_scheduler else "none",
    "dataset_mean": mean,
    "dataset_std": std,
    "num_classes": data_module.num_classes,
    "checkpoint_path": str(best_checkpoint.name)
}

with open(dataset_dir / "training_summary.json", "w") as f:
    json.dump(summary, f, indent=2)
print(f"✓ Saved training summary")

# Copy ALL TensorBoard logs (full logs for offline review)
tb_logs_src = Path(system_config.output_dir) / "kenyan_food_logs"
if tb_logs_src.exists():
    tb_logs_dest = dataset_dir / "tensorboard_logs"

    print(f"Copying TensorBoard logs from {tb_logs_src}...")
    shutil.copytree(tb_logs_src, tb_logs_dest, dirs_exist_ok=True)

    # Calculate total size
    total_size = sum(f.stat().st_size for f in tb_logs_dest.rglob('*') if f.is_file())
    print(f"✓ Saved TensorBoard logs ({total_size / 1_000_000:.1f} MB)")
else:
    print("⚠ No TensorBoard logs found")

# Create a ZIP file for easy download
print("\nCreating ZIP file...")
zip_path = Path("/kaggle/working/kenyan_food_model_output.zip")
if zip_path.exists():
    zip_path.unlink()

shutil.make_archive(
    str(zip_path.with_suffix('')),  # Remove .zip, make_archive adds it
    'zip',
    dataset_dir
)

zip_size = zip_path.stat().st_size / 1_000_000
print(f"✓ Created ZIP file: {zip_path.name} ({zip_size:.1f} MB)")

print(f"\n{'='*60}")
print(f"Output saved to: {dataset_dir}")
print(f"ZIP file: {zip_path}")
print(f"{'='*60}")
print("\nWhat's saved:")
print(f"  - best_model.ckpt (trained model)")
print(f"  - training_summary.json (metrics & config)")
print(f"  - tensorboard_logs/ (full logs for offline review)")
print("\nDownload options:")
print("  Option 1: Download ZIP directly from Output tab")
print("    → Look for 'kenyan_food_model_output.zip'")
print("    → Click to download (single file, faster)")
print("\n  Option 2: Create as Kaggle Dataset")
print("    1. Click 'Save Version' in Kaggle (top right)")
print("    2. After run completes, go to 'Output' tab")
print("    3. Click 'New Dataset' and name it (e.g., 'kenyan-food-model-v1')")
print("    4. Reusable in other notebooks!")
print("\nTo view TensorBoard logs locally:")
print("  1. Extract the ZIP file")
print("  2. Run: tensorboard --logdir=kenyan_food_model_output/tensorboard_logs")
print("  3. Open: http://localhost:6006")
print(f"{'='*60}")

## Step 8: Generate Predictions (TODO)

Add your inference code here to generate submission.csv

In [ ]:
# TODO: Load test data and generate predictions
# TODO: Create submission.csv
pass